In [15]:
import logzero
from logzero import logger
import config
from energyplus_wrapper import EPlusRunner
from eppy import modeleditor
from eppy.modeleditor import IDF

In [16]:
EPWFILE = "./model/CHAMBERY.epw"
building="./model/IDM.idf"
IDDPATH = "C:\EnergyPlusV9-3-0\Energy+.idd"
EPLUSPATH = "C:\EnergyPlusV9-3-0"
simu_name = 'Maison' 
IDF.setiddname(IDDPATH)
epmodel = IDF(building, EPWFILE)


In [17]:
logger.debug("running energyplus")
runner = EPlusRunner(EPLUSPATH)

simulation = runner.run_one(epmodel, EPWFILE)
                            
result=simulation.time_series

[D 210625 23:50:23 <ipython-input-17-01021ee67260>:1] running energyplus
[D 210625 23:50:23 runner:195] (Path('C:\\Temp\\energyplus_run_co8c1xuh\\eppy_idf.idf'), Path('c:\\Users\\elkhatts\\Desktop\\transfere\\optimisation\\NSGAii\\model\\CHAMBERY.epw'), Path('C:\\Temp\\energyplus_run_co8c1xuh'))
C:\Programmes\Python39\lib\site-packages\energyplus_wrapper\utils.py:84: UserWarning: Unable to parse csv file C:\Temp\energyplus_run_co8c1xuh\eplus-table.csv. Return raw string as fallback.
  warnings.warn(


In [18]:
print(result)

{'table': 'Program Version:,EnergyPlus, Version 9.3.0-baff08990c, YMD=2021.06.25 23:50\nTabular Output Report in Format: ,Comma\n\nBuilding:,IDM\nEnvironment:,ANNEE ** CHAMBERY - - MN7 WMO#=999\n\n----------------------------------------------------------------------------------------------------\nREPORT:,Annual Building Utility Performance Summary\nFOR:,Entire Facility\nValues gathered over      8760.00 hours\n\nSite and Source Energy\n\n,,Total Energy [kWh],Energy Per Total Building Area [kWh/m2],Energy Per Conditioned Building Area [kWh/m2]\n,Total Site Energy,2695.46,13.82,27.65\n,Net Site Energy,2695.46,13.82,27.65\n,Total Source Energy,8835.25,45.31,90.62\n,Net Source Energy,8835.25,45.31,90.62\n\n\nSite to Source Energy Conversion Factors\n\n,,Site=>Source Conversion Factor\n,Electricity,3.167\n,Natural Gas,1.084\n,District Cooling,1.056\n,District Heating,3.613\n,Steam,0.300\n,Gasoline,1.050\n,Diesel,1.050\n,Coal,1.050\n,Fuel Oil #1,1.050\n,Fuel Oil #2,1.050\n,Propane,1.050\n,O

In [22]:
def overheating(result):
    logger.debug("computing overheating")
    indoor = None
    out = None
    heures_inconfort=[]
    oh = []
    for data in result:
            if "eplus" in data:
                
                logger.debug("found eplus.csv")
                indoor = result[data].iloc[:, [
                    "Mean Air Temperature" in col for col in result[data].columns]]
                out = result[data].iloc[:,[
                    "Outdoor Air Drybulb Temperature" in col for col in result[data].columns]]
                Text_moy_jour=[float(out[i:289+i].mean()) for i in range(0,len(out),288)]
                Text_glissantes=moyenne_glissante(Text_moy_jour, 7)#moyenne glissante sur 7 jours
                Tconfort=[0.33*Tmoyext+18.8 for Tmoyext in Text_glissantes]
                print(Tconfort)    
                for zone, area in config.zones_areas.items():
                    print(zone)
                    oh_zone=0
                    heures_inconfort_zone=0
                    indoor_zone=indoor.iloc[:,[zone in col for col in indoor.columns]]
                    T_moy_jour=[float(indoor_zone[i:289+i].mean()) for i in range(0,len(indoor_zone),288)]
                    print(T_moy_jour)
                    for i in range(len(T_moy_jour)):
                        if T_moy_jour[i]>(Tconfort[i]+2):
                            oh_zone+=T_moy_jour[i]-(Tconfort[i]+2)
                            heures_inconfort_zone+=1
                    oh.append(oh_zone)
                    heures_inconfort.append(heures_inconfort_zone)
    area_tot=0
    areas=[]
    for zone,area in config.zones_areas.items():
        area_tot+=area
        areas.append(area)
    oh_tot=sum([x*y for x,y in zip(areas,oh)])/area_tot  #somme pondérée par les surfaces
    heures_inconfort_tot=sum([x*y for x,y in zip(areas,heures_inconfort)])/area_tot  
    logger.debug("overheating = %s °C/h" % (oh_tot))
    logger.debug("heures inconfort = %s " % (heures_inconfort_tot))
    return heures_inconfort_tot



In [20]:
overheating(result)

[D 210625 23:50:55 <ipython-input-19-d3489e20b940>:2] computing overheating
[D 210625 23:50:55 <ipython-input-19-d3489e20b940>:10] found eplus.csv


NameError: name 'moyenne_glissante' is not defined

In [6]:
def temp_glissante(result):
    indoor = None
    out = None
    for data in result:
            if "eplus" in data:
                logger.debug("found eplus.csv")
                out = result[data].iloc[:,
                                ["Outdoor Air Drybulb Temperature" in col for col in result[data].columns]]
                indoor = result[data].iloc[:, [
                    "Mean Air Temperature" in col for col in result[data].columns]]
                for index in indoor.columns:
                    for temp in indoor[index]:
                        Tmoyext=out.iat[indoor[index].values.tolist().index(temp), 0]
                        Tconfort=0.31*Tmoyext+17.8
                        print(Tmoyext,Tconfort)
                           


In [21]:
def moyenne_glissante(valeurs, intervalle):
    indice_debut = (intervalle - 1) // 2
    liste_moyennes = [sum(valeurs[i - indice_debut:i + indice_debut + 1]) / intervalle for i in range(indice_debut, len(valeurs) - indice_debut)]
    return liste_moyennes
 
valeurs = [1, 5, 8, 9, 4, 45, 85, 100, 85, 96]
intervalle = 5
 
print(valeurs)
print(moyenne_glissante(valeurs, intervalle))

[1, 5, 8, 9, 4, 45, 85, 100, 85, 96]
[5.4, 14.2, 30.2, 48.6, 63.8, 82.2]


In [155]:
for data in result:
    if "eplus" in data:
         indoor = result[data].iloc[:, ["Mean Air Temperature" in col for col in result[data].columns]]

In [156]:
indoor

,RDC THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep),VS THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep),COMBLE THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep),ZCH1:Zone Mean Air Temperature [C](TimeStep),ZCH2:Zone Mean Air Temperature [C](TimeStep),ZCH3:Zone Mean Air Temperature [C](TimeStep),ZSDB:Zone Mean Air Temperature [C](TimeStep)
0,21.655507,5.225721,4.389427,22.664373,22.987606,23.060229,22.184568
1,21.632581,5.217833,4.325159,22.667205,22.981317,23.052258,22.176662
2,21.609024,5.208724,4.262115,22.668895,22.974089,23.043364,22.168909
3,21.585316,5.198915,4.202123,22.669721,22.966233,23.033871,22.161322
4,21.561746,5.188769,4.146204,22.669890,22.957961,23.024002,22.153907
...,...,...,...,...,...,...,...
105115,15.712140,4.645237,0.376467,21.000000,21.000000,21.000000,16.213536
105116,15.625397,4.640803,0.327642,21.000000,21.000000,21.000000,16.206204
105117,15.569734,4.636654,0.280729,21.000000,21.000000,21.000000,16.200565
105118,15.528287,4.632697,0.235495,21.000000,21.000000,21.000000,16.196119


In [157]:
for data in result:
    if "eplus" in data:
         out = result[data].iloc[:,["Outdoor Air Drybulb Temperature" in col for col in result[data].columns]]

In [44]:
moyenne_jour1=out[0:288].mean()
print(moyenne_jour1)

Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)    9.125
dtype: float64


In [67]:
from statistics import mean

In [159]:
Temp=[]
for i in range (288):
    Temp.append(float(out.iloc[i]))
print(mean(Temp))

9.125


In [160]:
len(out)

105120

In [161]:
Text_moy_jour2=[float(out[i:289+i].mean()) for i in range(0,len(out),288)]

In [162]:
def moyenne_glissante(valeurs, intervalle):
    indice_debut = (intervalle - 1) // 2
    liste_moyennes=valeurs[:6]
    liste_moyennes += [sum(valeurs[i - indice_debut:i + indice_debut + 1]) / intervalle for i in range(indice_debut, len(valeurs) - indice_debut)]
    return liste_moyennes
 
valeurs = Text_moy_jour2
intervalle = 7
Text_glissantes=moyenne_glissante(valeurs, intervalle)

In [179]:
indoor_RDC=indoor["RDC THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep)"]
indoor_RDC

0         21.655507
1         21.632581
2         21.609024
3         21.585316
4         21.561746
            ...    
105115    15.712140
105116    15.625397
105117    15.569734
105118    15.528287
105119    15.494309
Name: RDC THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep), Length: 105120, dtype: float64

In [178]:
indoor_rdc_iloc=indoor.iloc[:,["RDC" in col for col in indoor.columns]]
indoor_rdc_iloc

,RDC THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep)
0,21.655507
1,21.632581
2,21.609024
3,21.585316
4,21.561746
...,...
105115,15.712140
105116,15.625397
105117,15.569734
105118,15.528287


In [173]:
indoor_RDC

0         21.655507
1         21.632581
2         21.609024
3         21.585316
4         21.561746
            ...    
105115    15.712140
105116    15.625397
105117    15.569734
105118    15.528287
105119    15.494309
Name: RDC THERMAL ZONE:Zone Mean Air Temperature [C](TimeStep), Length: 105120, dtype: float64

In [182]:
TRDC_moy_jour=[float(indoor_RDC[i:289+i].mean()) for i in range(0,len(indoor_RDC),288)]
TRDC_moy_jour

[22.774008509262163,
 21.08484489319485,
 20.355336531003967,
 19.432407355354332,
 19.232731223625994,
 19.1212442283873,
 18.745482617719738,
 18.233900609543152,
 17.016372620680347,
 16.39564679202268,
 16.645445156426423,
 16.397912934547794,
 16.368083040267123,
 17.34457930895343,
 17.45454494249345,
 16.975341319351365,
 17.330994596968136,
 16.918522015587037,
 16.65719348479712,
 16.545267725384996,
 17.49264005438181,
 17.720408412072306,
 17.642232826360168,
 17.484057292543373,
 17.769148960908325,
 18.748303257813816,
 18.212677707666376,
 18.023413615434553,
 17.934425438275444,
 17.19595543877702,
 17.165141553438,
 16.943298273748315,
 16.909419561557957,
 16.703164293892204,
 17.47372914028508,
 17.60861069054775,
 17.105862281587186,
 17.646447940952648,
 17.25214839682876,
 16.431733816484154,
 16.240319021160232,
 17.368860582006516,
 17.444224922154095,
 16.539790133369785,
 16.36134390801109,
 16.739673975656903,
 17.001657055931606,
 16.894928779947644,
 17.6617

In [181]:
TRDC_moy_jour=[float(indoor_rdc_iloc[i:289+i].mean()) for i in range(0,len(indoor_rdc_iloc),288)]
TRDC_moy_jour

[22.774008509262163,
 21.08484489319485,
 20.355336531003967,
 19.432407355354332,
 19.232731223625994,
 19.1212442283873,
 18.745482617719738,
 18.233900609543152,
 17.016372620680347,
 16.39564679202268,
 16.645445156426423,
 16.397912934547794,
 16.368083040267123,
 17.34457930895343,
 17.45454494249345,
 16.975341319351365,
 17.330994596968136,
 16.918522015587037,
 16.65719348479712,
 16.545267725384996,
 17.49264005438181,
 17.720408412072306,
 17.642232826360168,
 17.484057292543373,
 17.769148960908325,
 18.748303257813816,
 18.212677707666376,
 18.023413615434553,
 17.934425438275444,
 17.19595543877702,
 17.165141553438,
 16.943298273748315,
 16.909419561557957,
 16.703164293892204,
 17.47372914028508,
 17.60861069054775,
 17.105862281587186,
 17.646447940952648,
 17.25214839682876,
 16.431733816484154,
 16.240319021160232,
 17.368860582006516,
 17.444224922154095,
 16.539790133369785,
 16.36134390801109,
 16.739673975656903,
 17.001657055931606,
 16.894928779947644,
 17.6617

In [167]:
Tconfort=[0.33*Tmoyext+18.8 for Tmoyext in Text_glissantes]

In [168]:
def oh_RDC(TRDC,Tconfort):
    oh=0
    heures_inconfort=0
    for i in range(len(TRDC)):
        if TRDC[i]>(Tconfort[i]+2):
            oh+=TRDC[i]-(Tconfort[i]+2)
            heures_inconfort+=1
    return oh, heures_inconfort

In [141]:
print(oh_RDC(TRDC_moy_jour,Tconfort))

(0, 0)


In [172]:
for i in range(3):
    print(i,TRDC_moy_jour[i],Tconfort[i])

0 22.774008509262163 21.80966089965398
1 21.08484489319485 19.998790657439447
2 20.355336531003967 20.23189359861592


In [209]:
zones_areas={"RDC THERMAL ZONE":48.75,"ZCH1":14.86,"ZCH2":10.16,"ZCH3":9.64,"ZSDB":14.09}

In [150]:
area_tot=0
for zone,area in zone_areas.items():
    area_tot+=area
print(area_tot)

97.5


In [238]:
def moyenne_glissante(valeurs, intervalle):
    indice_debut = (intervalle - 1) // 2
    liste_moyennes=valeurs[:6]
    liste_moyennes += [sum(valeurs[i - indice_debut:i + indice_debut + 1]) / intervalle for i in range(indice_debut, len(valeurs) - indice_debut)]
    return liste_moyennes

def overheating(result):
    logger.debug("computing overheating")
    indoor = None
    out = None
    heures_inconfort=[]
    oh = []
    for data in result:
            if "eplus" in data:
                logger.debug("found eplus.csv")
                indoor = result[data].iloc[:, [
                    "Mean Air Temperature" in col for col in result[data].columns]]
                out = result[data].iloc[:,[
                    "Outdoor Air Drybulb Temperature" in col for col in result[data].columns]]
                Text_moy_jour=[float(out[i:289+i].mean()) for i in range(0,len(out),288)]
                Text_glissantes=moyenne_glissante(Text_moy_jour, 7)#moyenne glissante sur 7 jours
                Tconfort=[0.33*Tmoyext+18.8 for Tmoyext in Text_glissantes]    
                for zone, area in zones_areas.items():
                    print(zone)
                    oh_zone=0
                    heures_inconfort_zone=0
                    indoor_zone=indoor.iloc[:,[zone in col for col in indoor.columns]]
                    T_moy_jour=[float(indoor_zone[i:289+i].mean()) for i in range(0,len(indoor_zone),288)]
                    for i in range(len(T_moy_jour)):
                        if T_moy_jour[i]>(Tconfort[i]+2):
                            oh_zone+=T_moy_jour[i]-(Tconfort[i]+2)
                            heures_inconfort_zone+=1
                    oh.append(oh_zone)
                    heures_inconfort.append(heures_inconfort_zone)
    area_tot=0
    areas=[]
    for zone,area in zone_areas.items():
        area_tot+=area
        areas.append(area)
    oh_tot=sum(list(map(lambda x,y: x*y/area_tot ,areas,oh)))  #somme pondérée par les surfaces
    heures_inconfort_tot=sum(list(map(lambda x,y: x*y/area_tot ,areas,heures_inconfort))) 
    logger.debug("overheating = %s °C/h" % (oh_tot))
    logger.debug("heures inconfort = %s " % (heures_inconfort))
    return oh_tot, heures_inconfort_tot

In [239]:
overheating(result)

[D 210622 22:09:13 <ipython-input-238-37ec3c7fe94f>:8] computing overheating
[D 210622 22:09:13 <ipython-input-238-37ec3c7fe94f>:15] found eplus.csv
RDC THERMAL ZONE
ZCH1
ZCH2
ZCH3
ZSDB
[D 210622 22:09:15 <ipython-input-238-37ec3c7fe94f>:42] overheating = 752.6109871306319 °C/h
[D 210622 22:09:15 <ipython-input-238-37ec3c7fe94f>:43] heures inconfort = [172, 194, 178, 180, 189] 


(752.6109871306319, 179.2259487179487)

In [14]:
def moyenne_glissante_norme (valeurs, intervalle):
    indice_debut=(intervalle - 1) // 2
    liste_moyennes=valeurs[1:intervalle]
    liste_moyennes += [(0.2*valeurs[i - indice_debut]+0.3*valeurs[i - indice_debut+1]+0.4*valeurs[i - indice_debut+2]+
                    0.5*valeurs[i - indice_debut+3]+0.6*valeurs[i - indice_debut+4]+0.8*valeurs[i - indice_debut+5]+
                    valeurs[i - indice_debut+6]) / 3.8 for i in range(indice_debut, len(valeurs) - indice_debut)]
    return liste_moyennes

In [15]:
valeurs=[1,2,3,4,5,6,7,8,9,10,11]
intervalle=7
liste_moyennes=moyenne_glissante_norme (valeurs, intervalle)
print(liste_moyennes)

[2, 3, 4, 5, 6, 7, 4.947368421052632, 5.947368421052632, 6.947368421052632, 7.947368421052632, 8.947368421052632]


In [21]:
import random
LOW_WALLS, UP_WALLS = 10, 50 # épaisseurs en cm 
# Bounds windows
LOW_WINDOW, UP_WINDOW = 0, 3 # 4 types de fenetres
ranges = [(LOW_WALLS, UP_WALLS), (LOW_WALLS, UP_WALLS),
          (LOW_WALLS, UP_WALLS), (LOW_WINDOW, UP_WINDOW)]
nparam = len(ranges)
for p in ranges[0:nparam-1]:
    print (random.randrange(*p,5))

45
15
15
